# 级联分类器处理脚本
- 利用albumentations对图像进行增强、剪裁

In [1]:
from __future__ import print_function, absolute_import
import albumentations as A
import os.path as osp
import cv2
import glob
import re
from matplotlib import pyplot as plt
import random

In [2]:
class DataProcessor:
    def __init__(self, input_path, output_path):
        self.input_path = input_path
        self.output_path = output_path
        self.img_paths = glob.glob(osp.join(input_path, '*.jpg'))
        self.transform = A.Compose([
            A.OneOf([
                A.IAAAdditiveGaussianNoise(),
                A.GaussNoise(),
                A.MotionBlur(p=0.2)
            ], p=0.4),
            A.OneOf([
                A.OpticalDistortion(p=0.3),
                A.GridDistortion(p=.1),
                A.IAAPiecewiseAffine(p=0.3),
            ], p=0.3),
            A.OneOf([
                A.CLAHE(clip_limit=2),
                A.IAASharpen(),
                A.IAAEmboss(),
                A.RandomBrightnessContrast(),
            ], p=0.3),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Resize(height=64, width=64)
        ])


    def cutAndAugment(self):
        num = 0

        for img_path in self.img_paths:
            image = cv2.imread(img_path)
            r = cv2.selectROI('roi',image,False,False)
            image_roi = image[int(r[1]):int(r[1]+r[3]),int(r[0]):int(r[0]+r[2])]
            image_roi = cv2.cvtColor(image_roi, cv2.COLOR_BGR2RGB)
            # 一张图增强为5张图
            for i in range(5):
                num += 1
                transformed = self.transform(image=image_roi)
                image_trans = transformed["image"]
                image_trans = cv2.cvtColor(image_trans, cv2.COLOR_RGB2GRAY)
                outname = self.output_path + str(num) + ".jpg"
                cv2.imwrite(outname, image_trans)

In [3]:
processor = DataProcessor(input_path="/home/qianchen/ssd/workspace/code/image/",output_path="/home/qianchen/ssd/workspace/code/DataAfter/")
processor.cutAndAugment()

In [2]:
class DataProcessorNeg:
    def __init__(self, input_path, output_path):
        self.input_path = input_path
        self.output_path = output_path
        self.img_paths = glob.glob(osp.join(input_path, '*.jpg'))
        self.transform = A.Compose([
            A.OneOf([
                A.IAAAdditiveGaussianNoise(),
                A.GaussNoise(),
                A.MotionBlur(p=0.2)
            ], p=0.5),
            A.OneOf([
                A.OpticalDistortion(p=0.3),
                A.GridDistortion(p=.1),
                A.IAAPiecewiseAffine(p=0.3),
            ], p=0.5),
            A.OneOf([
                A.CLAHE(clip_limit=2),
                A.IAASharpen(),
                A.IAAEmboss(),
                A.RandomBrightnessContrast(),
            ], p=0.5),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Resize(height=640, width=520)
        ])


    def cutAndAugment(self):
        num = 0
        for img_path in self.img_paths:
            image = cv2.imread(img_path)
            r = cv2.selectROI('roi',image,False,False)
            image_roi = image[int(r[1]):int(r[1]+r[3]),int(r[0]):int(r[0]+r[2])]
            image_roi = cv2.cvtColor(image_roi, cv2.COLOR_BGR2RGB)
            # 一张图增强为5张图
            for i in range(6):
                num += 1
                transformed = self.transform(image=image_roi)
                image_trans = transformed["image"]
                image_trans = cv2.cvtColor(image_trans, cv2.COLOR_RGB2GRAY)
                outname = self.output_path + str(num) + ".jpg"
                cv2.imwrite(outname, image_trans)

In [ ]:
processor = DataProcessorNeg(input_path="/media/qianchen/My Passport/image",output_path="/home/qianchen/ssd/workspace/code/trainCascadeInfantry/neg/")
processor.cutAndAugment()